In [1]:
# ! which python

In [2]:
# ! uv run main.py SimAM_call --data-type "sst2" --seed 30

In [3]:
import sys
from torch.nn.utils.rnn import pad_sequence
import transformers
import torch
# sys.path.append("./Incontext-learning") # this part works for goole colab

In [4]:
import utility
import metric
import importlib

importlib.reload(utility)
importlib.reload(metric)
from utility import (
    data_selection,
    evaluate_zeroshot,
    evaluate_finetuning,
    evaluate_demonstrations as evaluate_fewshots,
    finetune_model_eval,
    extract_attn_weights,
)
from metric import Rec2FTP, SimAOU, SimAM, Kendall
from torch.utils.data import DataLoader

In [5]:
# uncomment below if you want to use customized attention module for attention weights extraction before softmax
# from transformers.models.qwen3 import modeling_qwen3
# from customized_Qwen import Qwen3Attention_v1, AttentionWeightsCollector

# modeling_qwen3.Qwen3Attention = Qwen3Attention_v1

# check point

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load SST-2 dataset
dataset = load_dataset("glue", "sst2")

# load SST-5 dataset
# dataset = load_dataset("SetFit/sst5")

# Load Qwen3 tokenizer and model
model_name = "Qwen/Qwen3-0.6B"
model_path = "/home/students/wli/UniHeidelberg/semster2/final_projects/models/Qwen3-0.6B-Base"
# model_path = model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map = "auto", attn_implementation="eager")  # Specify eager for attention before softmax

tokenizer.padding_side = "left"
# Make sure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# if getattr(model.config.pad_token_id, None) is None:
    # pass

In [7]:
train_dataset = dataset["train"] # the orginal paper use this
test_dataset = dataset["validation"]

def add_idx(example, idx):
    example["idx"] = idx
    return example

test_dataset = test_dataset.map(add_idx, with_indices=True)
# Format examples as causal LM inputs
def preprocess_function(examples):
    label_map = {0: "negative", 1: "positive"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["sentence"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

def preprocess_function_sst5(examples):
    label_map = {0: "terrible", 1: "bad", 2: "neutral", 3: "good", 4: "very great"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["text"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


# Tokenize the dataset
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
# train_tokenized_datasets = train_dataset.map(preprocess_function_sst5, batched = True, remove_columns = train_dataset.column_names)
# test_tokenized_datasets = test_dataset.map(preprocess_function_sst5, batched=True, remove_columns=test_dataset.column_names)
# Data collator (handles padding dynamically in batch)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# test Kendall

In [8]:
len(test_dataset)
test_dataset2 = test_dataset.select(range(20))  # for testing purpose, use only 100 samples

In [9]:
out = Kendall(model, tokenizer,train_dataset,test_dataset, data_type="sst2")  # for testing purpose, use only 100 samples

100%|██████████| 872/872 [08:03<00:00,  1.80it/s]


Kendall's tau of sst2 is 0.1873693018, random baseline is 0.0000711622


In [10]:
out

np.float64(0.18296876693546485)

In [20]:
len(out[3])

28

In [22]:
out[3][0].shape

torch.Size([20, 464])

In [21]:
out[2][0].shape

torch.Size([464, 20])

In [19]:
out[1][0][15].shape

torch.Size([4, 39, 2048])

In [17]:
out[0][0][0].shape

torch.Size([1, 464, 2048])

In [7]:
# attention_collector = AttentionWeightsCollector(model)
# attention_collector.enable_collection()

In [8]:
# # import Qwne3ForCausalLM
# from transformers import Qwen3ForCausalLM
# model = Qwen3ForCausalLM.from_pretrained(model_path, device_map = "auto")

In [9]:
# output = extract_attentionweights(
#     model, tokenizer, ["This is a test sentence."," Another test sentence. label: "], 
#     batch_size=2, before_softmax=True
# )

In [10]:
# output = extract_raw_attention_scores(
#     model, tokenizer, ["This is a test sentence."," Another test sentence. label: "], 
#     batch_size=1
# )

In [12]:
# test the behavior of LLM before fine tuing
prompt = "Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label:"
prompts = ["test prompt how old are ", "test prompt 2 hello world"]

inputs = tokenizer(prompts, return_tensors="pt").to(model.device)
# You can control max_length or early stopping
output_ids = model.generate(
    **inputs,
    max_new_tokens=1,  # adjust based on expected label length
    do_sample=False,  # greedy decoding
    pad_token_id=tokenizer.pad_token_id,
)


# Decode generated tokens
generated_text = tokenizer.decode(output_ids[1], skip_special_tokens=True)
print(generated_text)

test prompt 2 hello world



# test SimAM

In [ ]:
sim_scores_ft, sim_scores_bf_ft = SimAM(
    model,
    tokenizer,
    train_dataset,
    test_dataset,
    best_seed=30,
    data_type="sst2",
    num_data_points=32,
)

  3%|▎         | 28/872 [00:07<03:59,  3.52it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x14689a95db20>>
Traceback (most recent call last):
  File "/home/students/wli/UniHeidelberg/semster2/final_projects/incontext_learning/dual-form-attention/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 796, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                      ^^^^^^^^^^^^
KeyboardInterrupt: 
  8%|▊         | 70/872 [00:19<03:48,  3.52it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x14689a95db20>>
Traceback (most recent call last):
  File "/home/students/wli/UniHeidelberg/semster2/final_projects/incontext_learning/dual-form-attention/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  9%|▉         | 78/872 [00:22<03:45,  3.53it/s]

In [ ]:
# test_dataset = test_dataset.select(range(20))
out = SimAM(
    model,
    tokenizer,
    train_dataset,
    test_dataset,
    data_type="sst2",
    batch_size_icl=1,
    batch_size_zsl=1,
)

  2%|▏         | 20/872 [00:06<04:40,  3.04it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x14ac9cb2dac0>>
Traceback (most recent call last):
  File "/home/students/wli/UniHeidelberg/semster2/final_projects/incontext_learning/dual-form-attention/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  7%|▋         | 59/872 [00:19<04:26,  3.04it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x14ac9cb2dac0>>
Traceback (most recent call last):
  File "/home/students/wli/UniHeidelberg/semster2/final_projects/incontext_learning/dual-form-attention/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  8%|▊         | 66/872 [00:22<04:24,  3.04it/s]

In [ ]:
out[0][0][3].shape

In [ ]:
1 + 1

In [ ]:
# attention_collector.collect_weights()

In [ ]:
# a = attention_collector.get_weights_for_layer(4)
# b = attention_collector.get_all_weights()

In [ ]:
# a.shape

In [ ]:
# test_dataset = test_dataset.select(range(32)) # use a smaller test set for debugging

In [ ]:
# for batch in DataLoader(test_dataset, batch_size=2, shuffle=True):
#     print(batch)
#     print(type(batch))
#     break

In [ ]:

a = SimAOU(model,tokenizer,train_dataset, test_dataset, best_seed = 0, num_data_points = 32, data_type = "sst5", batch_size_icl = 32, batch_size_zsl = 16)

In [ ]:
a

In [ ]:
# finetune_model_eval(model, tokenizer,train_dataset,test_dataset, best_seed = 50, data_type="sst5", num_data_points = 32)